# Advent of code 2021

My first time entering Advent of Code. I'm pretty rusty in Python at the moment, so I have shameless copied the utility functions from Peter Norvieg's [2020 solutions](https://github.com/norvig/pytudes/blob/main/ipynb/Advent-2020.ipynb) so that my attempts will at least start off in a well structured fashion.

Only entering to complete, not to get a fast time.

In [88]:
from __future__  import annotations
from collections import Counter, defaultdict, namedtuple, deque
from itertools   import permutations, combinations, product, chain, tee
from functools   import lru_cache
from typing      import Dict, Tuple, Set, List, Iterator, Optional, Union, NamedTuple

import operator
import math
import ast
import sys
import re

Peter Norveigs utilities:

In [89]:
def data(day: int, parser=str, sep='\n') -> list:
    "Split the day's input file into sections separated by `sep`, and apply `parser` to each."
    sections = open(f'data/input{day}.txt').read().rstrip().split(sep)
    return [parser(section) for section in sections]
     
def do(day, *answers) -> Dict[int, int]:
    "E.g., do(3) returns {1: day3_1(in3), 2: day3_2(in3)}. Verifies `answers` if given."
    g = globals()
    got = []
    for part in (1, 2):
        fname = f'day{day}_{part}'
        if fname in g: 
            got.append(g[fname](g[f'in{day}']))
            if len(answers) >= part: 
                assert got[-1] == answers[part - 1], (
                    f'{fname}(in{day}) got {got[-1]}; expected {answers[part - 1]}')
    return got

def atoms(text: str, ignore=r'', sep=None) -> Tuple[Union[int, str]]:
    "Parse text into atoms (numbers or strs), possibly ignoring a regex."
    if ignore:
        text = re.sub(ignore, '', text)
    return tuple(map(atom, text.split(sep)))

def atom(text: str) -> Union[float, int, str]:
    "Parse text into a single float or int or str."
    try:
        val = float(text)
        return round(val) if round(val) == val else val
    except ValueError:
        return text

My utilities

In [90]:
def pairwise(iterable): # Defined in itertools in python 3.10 but I'm in 3.9
    "pairwise('ABCDEFG') --> AB BC CD DE EF FG"
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

### Day 1: Sonar Sweep
Part 1 is to compute the number of times we see an increase between elements. Itertools has a helpful _pairwise_ function that allows comparison between two adjacent elements in a list without making a full copy. Part 2 is to compute a sliding window first.. so I 'adapted' the _pairwise_ function to allow a 3-element sliding window but it's not nicely generalised to different window lengths.

In [91]:
in1: List[int] = data(1, int)

In [92]:
def day1_1(nums):
    return sum(second > first for first,second in pairwise(nums))

In [93]:
def day1_2(nums):
    "Niave extension of itertools.pairwise"
    a, b, c = tee(nums, 3)
    next(b, None)
    next(c, None)
    next(c, None)
    moving_average = [x+y+z for x, y, z in zip(a, b, c)]
    return day1_1(moving_average)

In [94]:
assert day1_1([199, 200, 208, 210, 200, 207, 240, 269, 260, 263]) == 7
assert day1_2([199, 200, 208, 210, 200, 207, 240, 269, 260, 263]) == 5

In [95]:
do(1, 1832, 1858)

[1832, 1858]

### Day 2: Dive!

For part 1 we don't need to worry about the order of the commands (assuming that we don't need to verify that the depth is always sensible). Brute fore approach for part 2.

In [96]:
Instruction = Tuple[str, int] # e.g. ('up', 1)
Program = List[Instruction]

in2: Program = data(2, atoms)

In [97]:
def sum_direction(program, direction):
    return sum(x[1] for x in filter(lambda x: x[0]==direction, program))

assert sum_direction([('down',2),('forward',7),('down',1)],"down") == 3

def day2_1(program):
    "Assumes that the input is valid and we don't have to worry about the submarine trying to go above the waterline"
    horizontal_position = sum_direction(program,"forward")
    depth = sum_direction(program,"down") - sum_direction(program,"up")
    return depth * horizontal_position

In [98]:
def day2_2(program):
    aim, depth, horizontal_position = 0, 0, 0
    for command, v in program:
        if command == "down":
            aim += v
        elif command == "up":
            aim -= v
        elif command == "forward":
            depth += aim * v
            horizontal_position += v
        else:
            raise Exception('Unexpected command %s',command)
    return depth * horizontal_position

In [99]:
do(2, 1484118, 1463827010)

[1484118, 1463827010]

### Day 3: Binary Diagnostic
Full brute force today. For part I I'm sure there must be a better way to unpack the most common element from a _Counter_ and to use the two's complement operator to compute epsilon. In part II, there must be a better way to find the most common element with a default.

I might revisit  these at a later date.

In [100]:
in3: List(int) = data(3, str)

In [101]:
def day3_1(nums):
    nums = [list(x) for x in nums]
    nums = list(map(list, zip(*nums)))
    counters = [Counter(x) for x in nums]
    gamma = "".join(str(x.most_common(1)[0][0]) for x in counters)
    epsilon = "".join(str(x.most_common()[-1][0]) for x in counters)
    return int(gamma,base=2) * int(epsilon, base=2)

def get_rating(nums, mode):
    ix = 0
    while len(nums) > 1 and ix < len(nums[0]):
        counter = Counter(x[ix] for x in nums)
        v = counter.most_common(1)[0] if mode=="most" else counter.most_common()[-1]
        if v[1]*2 == len(nums):
            v = '1' if mode=="most" else '0'
        else:
            v = v[0]
        nums = list(filter(lambda x: x[ix] == v,nums))
        ix += 1
    return nums[0]

def day3_2(nums):
    oxygen = get_rating(nums,mode="most")
    co2 = get_rating(nums,mode="least")
    return int(oxygen,base=2) * int(co2,base=2)

In [102]:
do(3, 2724524, 2775870)

[2724524, 2775870]

### Day 4: Giant Squid
I implemented a BingoBoard class on my first attempt here, but decided this was unnecessary when I revisited it. I think my runtime complexity is _O(NDB^2)_ for N boards, B elements per board and D draws because of the _is_bingo_ function, but the input is not that big.

In [103]:
def parse_bingo_boards(line: str) -> List[int]:
    return list(map(int,re.split("\W+",line.strip())))

in4: List[int] = data(4, parse_bingo_boards, sep='\n\n')

In [104]:
def is_bingo(hits, width = 5):
    out = any(all(hits[i:i+width]) for i in range(0,len(hits),width)) # check for complete rows
    if not out:
        out = any(all(hits[i:len(hits):width]) for i in range(width)) # check for complete columns
    return out

def score_bingo_board(board, draws):
    hits = [False for x in range(len(board))]
    bingo = False
    for i, draw in enumerate(draws):
        if draw in board:
            hits[board.index(draw)] = True
            bingo = is_bingo(hits)
            if bingo: break
    if bingo:
        return (i, draw * sum(x for x, h in zip(board,hits) if not h))
    else:
        return (math.inf, None)

In [105]:
def day4_1(boards):
    scores = [score_bingo_board(x, boards[0]) for x in boards[1:]]
    out = min(scores, key = lambda x: x[0])
    return out[1]

def day4_2(boards):
    scores = [score_bingo_board(x, boards[0]) for x in boards[1:]]
    scores = filter(lambda x: math.isfinite(x[0]), scores)
    out = max(scores, key = lambda x: x[0])
    return out[1]

In [106]:
do(4, 49860, 24628)

[49860, 24628]

### Day 5: Hydrothermal Venture
Part I is to count the number of overlapping points given a set of horizontal / vertical line segments. Pythons _Counter_ again was quite useful to count the number of overlaps. Part II just required an extension to _parse\_line\_segments_ to deal with diagonal lines.

In [107]:
def parse_line_segments(line: str) -> Tuple[int, int, int, int]:  
    a,b,c,d = map(int,re.findall(r'[^->,\s]+',line))
    assert a == c or b == d or (abs(a-c) == abs(b-d)), "Lines must be at 0, 45, 90 degrees"
    return (a,b,c,d)

in5: Tuple[int, int, int, int] = data(5, parse_line_segments)

In [108]:
def is_diagonal(segment):
    a,b,c,d = segment
    return (abs(a-c) == abs(b-d)) and not (a == c or b == d)

def expand_segment(segment):
    a,b,c,d = segment
    if a == c:
        x1, x2 = min(b,d), max(b, d)
        return [(a,x) for x in range(x1,x2+1)]
    elif b == d:
        x1, x2 = min(a,c), max(a, c)
        return [(x,b) for x in range(x1,x2+1)]
    else:
        signX = 1 if c > a else -1
        signY = 1 if d > b else -1
        return [(a+signX*i,b+signY*i) for i in range(abs(a-c)+1)]

In [109]:
def day5_1(segments):
    counter = Counter()
    for segment in segments:
        if not is_diagonal(segment):
            counter.update(expand_segment(segment))
    return sum(counter[x]>1 for x in counter)

def day5_2(segments):
    counter = Counter()
    for segment in segments:
        counter.update(expand_segment(segment))
    return sum(counter[x]>1 for x in counter)

In [110]:
do(5, 5167, 17604)

[5167, 17604]

### Day 6: Lanternfish
A question about modelling the population size of some fish, given that they reproduce every 6 days (or 8 if they are new), which lends itself nicely to recursion and the _lru\_cache_. The grid of cached values should be only 8 * 256 for part II.

In [111]:
in6: List[int] = data(6,int,sep=",")

In [112]:
@lru_cache
def population_size(fish_timer, days):
    step = min(days, fish_timer+1)
    if days == 0:
        out = 1
    elif days >= fish_timer+1:
        out = population_size(6, days - fish_timer - 1) + population_size(8, days - fish_timer - 1)
    else:
        out = population_size(fish_timer - days, 0)
    return out

assert population_size(3,3) == 1
assert population_size(3,4) == 2
assert population_size(3,10) == 2
assert population_size(3,11) == 3
assert population_size(3,13) == 4

In [113]:
def day6_1(fish_timers): return sum(population_size(x,80) for x in fish_timers)

def day6_2(fish_timers): return sum(population_size(x,256) for x in fish_timers)

In [114]:
do(6, 386755, 1732731810807)

[386755, 1732731810807]

### Day 7: The Treachery of Whales
Part I is to compute _min\_p \sum\_i abs(x\_i - p)_ and part II is to compute _min\_p \sum\_i abs(x\_i - p) * (abs(x\_i - p)+1)/2_ for a vector _x_ of initial positions. There is probably a clever way to solve these minimization problems, however the input is not that big so I opted for the brute force approach.

In [115]:
in7: List[int] = data(7,int,sep=",")

In [116]:
def day7_1(positions):
    fuel = math.inf
    for p in range(min(positions),max(positions)+1):
        v = sum(abs(x-p) for x in positions)
        fuel = min(v,fuel)
    return fuel

def day7_2(positions):
    fuel = math.inf
    for p in range(min(positions),max(positions)+1):
        v = sum((abs(x-p)*(abs(x-p)+1))//2 for x in positions)
        fuel = min(v,fuel)
    return fuel

In [117]:
do(7, 351901, 101079875)

[351901, 101079875]

### Day 8: Seven segment search
Labels for a 7 segment display are scrambled, and the task is to decode 4 numbers given an encoded list of numbers 0-9. Part I is relatively straight forward as you are only asked to decode numbers with a unique number of bars. For Part II I ended up with a reasonably verbose tree of if-statements to complete my 'decoder'. 

In [118]:
Display = NamedTuple("Display", [('pattern', List[frozenset]), ('output', List[frozenset])])

def parse_digits(line: str) -> Display:
    line = line.split("|")
    return Display([frozenset(x) for x in re.findall(r'[^\s]+',line[0])],[frozenset(x) for x in re.findall(r'[^\s]+',line[1])])

in8: List[Display] = data(8, parse_digits)

In [119]:
def day8_1(displays: List[Display]) -> int:
    counter = Counter()
    for display in displays:
        counter.update(map(len,display.output))
    return sum(counter[x] for x in [2,4,3,7])

In [120]:
def decode_display(display: Display) -> int:
    lens = {2:1,7:8,4:4,3:7}
    encoder = dict()
    for x in display.pattern:
        if len(x) in lens:
            encoder[lens[len(x)]] = x
    for x in display.pattern:
        if len(x) in lens:
            continue
        elif len(x) == 6 and encoder[4].issubset(x):
            encoder[9] = x
        elif len(x) == 6 and not encoder[1].issubset(x):
            encoder[6] = x
        elif len(x) == 6:
            encoder[0] = x
        elif len(x) == 5 and encoder[7].issubset(x):
            encoder[3] = x
        elif len(x) == 5 and len(x.intersection(encoder[4])) == 3:
            encoder[5] = x
        else:
            encoder[2] = x
    decoder = {v:k for k,v in encoder.items()}
    assert all(x in decoder for x in display.pattern)
    return int("".join([str(decoder[x]) for x in display.output]))


def day8_2(displays: List[Display]) -> int:
    return sum(map(decode_display, displays))


In [121]:
do(8, 387, 986034)

[387, 986034]